   # Kaggle Fashion-MNIST Classifier Using Keras

In this project a deep learning based classifier will be constructed using `Keras` and `Tensorflow`. This is part of the Deep Learning course of Kaggle. The dataset can be found [here](https://www.kaggle.com/zalando-research/fashionmnist). This dataset contains greyscale pictures of clothes from Zalando's storefront. It is comprised of a train set containing 60,000 pictures and a test set containing 10,000 pictures. Each picture is labeled to be one of 10 classes of clothes.

# Setup

Select GPU:

In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'
import os
os.environ['TF_MIN_GPU_MULTIPROCESSOR_COUNT']='4'
os.environ['CUDA_VISIBLE_DEVICES'] = '0,1'
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 14380945388489136212
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 9104897474
locality {
  bus_id: 1
  links {
  }
}
incarnation: 3341627468246877851
physical_device_desc: "device: 0, name: GeForce RTX 2080 Ti, pci bus id: 0000:26:00.0, compute capability: 7.5"
, name: "/device:GPU:1"
device_type: "GPU"
memory_limit: 3135687884
locality {
  bus_id: 1
  links {
  }
}
incarnation: 15350896844351988525
physical_device_desc: "device: 1, name: GeForce GTX 1050 Ti, pci bus id: 0000:25:00.0, compute capability: 6.1"
]


Imports

In [2]:
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow import keras
# import tensorflow as tf

Function to prepare the data:

In [3]:
# In the dataset each picture is 28x28 pixels
img_rows, img_cols = 28, 28
# There are 10 classes of clothing in the dataset
num_classes = 10

def prep_data(raw):
    '''Prepare data for use by the deep learning model
    Args:
        raw (numpy array): array containing the raw dataset to transform
    Returns:
        numpy array, numpy array: transformed datasets of data and labels ready for use by the ml model
    '''
    # separate and one-hot encode labels
    y = raw[:,0]
    out_y = keras.utils.to_categorical(y, num_classes)
    
    # cut out values, reshape them and normalize the color intensity values
    x = raw[:, 1:]
    num_images = raw.shape[0]
    out_x = x.reshape(num_images, img_rows, img_cols, 1)
    out_x = out_x / 255
    
    return out_x, out_y

fashion_file = "./fashion-mnist_train.csv"
fashion_data = np.loadtxt(fashion_file, skiprows = 1, delimiter = ',')
x, y = prep_data(fashion_data)

Neural network model:

In [4]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, Dropout
batch_size = 16

fashion_model = Sequential()
fashion_model.add(Conv2D(16, kernel_size = (3,3),
                        activation = 'relu',
                        input_shape = (img_rows, img_cols, 1)))
fashion_model.add(Conv2D(16, kernel_size = (3,3), activation = 'relu'))
fashion_model.add(Flatten())
fashion_model.add(Dense(128, activation = 'relu'))
fashion_model.add(Dense(num_classes, activation = 'softmax'))
fashion_model.compile(loss = keras.losses.categorical_crossentropy,
                     optimizer = 'adam',
                     metrics = ['accuracy'])
fashion_model.fit(x, y,
                 batch_size = batch_size,
                 epochs = 3,
                 validation_split = 0.2)


Train on 48000 samples, validate on 12000 samples
Epoch 1/3
48000/48000 [==============================] - 8s 167us/sample - loss: 0.4016 - accuracy: 0.8547 - val_loss: 0.3123 - val_accuracy: 0.8900
Epoch 2/3
48000/48000 [==============================] - 6s 134us/sample - loss: 0.2514 - accuracy: 0.9070 - val_loss: 0.2556 - val_accuracy: 0.9079
Epoch 3/3
48000/48000 [==============================] - 6s 134us/sample - loss: 0.1847 - accuracy: 0.9303 - val_loss: 0.2582 - val_accuracy: 0.9089


The model shows an accuracy of about 90 percent and takes approximately 45s to train and test on a GTX 1050 ti. Now in order to make the model more efficient some dropout layers will be added and the stride length will be increased. The first step will be to add dropout layers.
The main benefit of adding dropout layers are that they reduce overfitting in the model. This means that the model can be made more complex and hopefully achieve an even higher accuracy score without being limited by this phenomenon. 

In [5]:
batch_size = 16

fashion_model = Sequential()
fashion_model.add(Conv2D(16, kernel_size = (3,3),
                        activation = 'relu',
                        input_shape = (img_rows, img_cols, 1)))
fashion_model.add(Dropout(0.5))
fashion_model.add(Conv2D(16, kernel_size = (3,3), activation = 'relu'))
fashion_model.add(Dropout(0.5))
fashion_model.add(Flatten())
fashion_model.add(Dense(128, activation = 'relu'))
fashion_model.add(Dropout(0.5))
fashion_model.add(Dense(num_classes, activation = 'softmax'))
fashion_model.compile(loss = keras.losses.categorical_crossentropy,
                     optimizer = 'adam',
                     metrics = ['accuracy'])
fashion_model.fit(x, y,
                 batch_size = batch_size,
                 epochs = 3,
                 validation_split = 0.2)

Train on 48000 samples, validate on 12000 samples
Epoch 1/3
48000/48000 [==============================] - 7s 147us/sample - loss: 0.5806 - accuracy: 0.7898 - val_loss: 0.3620 - val_accuracy: 0.8698
Epoch 2/3
48000/48000 [==============================] - 7s 142us/sample - loss: 0.4181 - accuracy: 0.8481 - val_loss: 0.3204 - val_accuracy: 0.8860
Epoch 3/3
48000/48000 [==============================] - 7s 139us/sample - loss: 0.3761 - accuracy: 0.8625 - val_loss: 0.2986 - val_accuracy: 0.8912


### Results

The model with dropout layers showed no noticeable decrease in accuracy. The training time did increase slightly, however 

Next the stride length in the different convolutional layers will be increased.
Increasing the stride length of the convolutional layers means that the model training time should be reduced, with minimal impact on the resulting accuracy. The model will apply each of the convolutions less often and so will be less computationally expensive to train. Applying convolutions too often can lead to the model getting redundant information, which means that some of the calculation steps are unnecessary.

In [6]:
batch_size = 16

fashion_model = Sequential()
fashion_model.add(Conv2D(16, kernel_size = (3,3),
                        activation = 'relu',
                        input_shape = (img_rows, img_cols, 1)))
fashion_model.add(Dropout(0.5))
fashion_model.add(Conv2D(16, kernel_size = (3,3), strides = 3, activation = 'relu'))
fashion_model.add(Dropout(0.5))
fashion_model.add(Flatten())
fashion_model.add(Dense(128, activation = 'relu'))
fashion_model.add(Dropout(0.5))
fashion_model.add(Dense(num_classes, activation = 'softmax'))
fashion_model.compile(loss = keras.losses.categorical_crossentropy,
                     optimizer = 'adam',
                     metrics = ['accuracy'])
fashion_model.fit(x, y,
                 batch_size = batch_size,
                 epochs = 3,
                 validation_split = 0.2)

Train on 48000 samples, validate on 12000 samples
Epoch 1/3
48000/48000 [==============================] - 7s 139us/sample - loss: 0.6873 - accuracy: 0.7472 - val_loss: 0.4462 - val_accuracy: 0.8334
Epoch 2/3
48000/48000 [==============================] - 6s 131us/sample - loss: 0.5262 - accuracy: 0.8066 - val_loss: 0.3907 - val_accuracy: 0.8602
Epoch 3/3
48000/48000 [==============================] - 6s 130us/sample - loss: 0.4784 - accuracy: 0.8220 - val_loss: 0.3660 - val_accuracy: 0.8648


Making the model more complex to try to increase the accuracy:

In [10]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=(img_rows, img_cols, 1)))
model.add(Dropout(0.5))
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu'))
model.add(Dropout(0.5))
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu'))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer='adam',
              metrics=['accuracy'])
model.fit(x, y,
          batch_size=16,
          epochs=5,
          validation_split = 0.2)

Train on 48000 samples, validate on 12000 samples
Epoch 1/5
48000/48000 [==============================] - 8s 156us/sample - loss: 0.4588 - accuracy: 0.8338 - val_loss: 0.3254 - val_accuracy: 0.8867
Epoch 2/5
48000/48000 [==============================] - 7s 147us/sample - loss: 0.3060 - accuracy: 0.8859 - val_loss: 0.2755 - val_accuracy: 0.9011
Epoch 3/5
48000/48000 [==============================] - 7s 147us/sample - loss: 0.2587 - accuracy: 0.9029 - val_loss: 0.2647 - val_accuracy: 0.9065
Epoch 4/5
48000/48000 [==============================] - 7s 150us/sample - loss: 0.2240 - accuracy: 0.9137 - val_loss: 0.2656 - val_accuracy: 0.9041
Epoch 5/5
48000/48000 [==============================] - 7s 147us/sample - loss: 0.1995 - accuracy: 0.9242 - val_loss: 0.2668 - val_accuracy: 0.9084


In [8]:
model = Sequential()
model.add(Conv2D(128, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=(img_rows, img_cols, 1)))
model.add(Dropout(0.5))
model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer='adam',
              metrics=['accuracy'])
model.fit(x, y,
          batch_size=16,
          epochs=5,
          validation_split = 0.2)

Train on 48000 samples, validate on 12000 samples
Epoch 1/5
48000/48000 [==============================] - 12s 251us/sample - loss: 0.4109 - accuracy: 0.8513 - val_loss: 0.3107 - val_accuracy: 0.8860
Epoch 2/5
48000/48000 [==============================] - 12s 244us/sample - loss: 0.2806 - accuracy: 0.8964 - val_loss: 0.2761 - val_accuracy: 0.9031
Epoch 3/5
48000/48000 [==============================] - 12s 242us/sample - loss: 0.2297 - accuracy: 0.9134 - val_loss: 0.2472 - val_accuracy: 0.9093
Epoch 4/5
48000/48000 [==============================] - 12s 243us/sample - loss: 0.1963 - accuracy: 0.9254 - val_loss: 0.2502 - val_accuracy: 0.9109
Epoch 5/5
48000/48000 [==============================] - 12s 243us/sample - loss: 0.1673 - accuracy: 0.9362 - val_loss: 0.2685 - val_accuracy: 0.9072


## Results
Even with a much higher complexity and more training epochs, the model does not seem to improve much. 91% Accuracy seems to be a tough cap to break.

# Summary

In this project the MNIST-Fashion dataset containing images of clothing from Zalando was analyzed and a Keras model was constructed to classify the pictures into 10 categories of clothing. The model was optimized for speed and to combat overfitting, to allow for a more complex model. But in the end, even a much more complex model did not perform much better than the simple one. It might be interesting to see if the accuracy of the model could be improved by augmenting the training data to gain access to a larger dataset.